# Task:
Suppose you work for an electricity generation and retailing company which wants to setup microgrids within a rural region. The company has done its research and knows all the geo-coded customer locations (nodes) in the region. For the purpose of this assignment, we will simply consider the customer location as points pi = (xi, yi) in two-dimensional space R2 with no further information attached to them.

## Question 1:
Using the ADT class for points in 2-dimensional Euclidean space from before, implement a function to generate a test set of n random nodes/points, where n is a user-definable parameter.

In [1]:
# define the Point ADT class to constuct point wchich will be used later.
from math import *
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"
    
    def setPoint(self, newx, newy):
        self.x = newx
        self.y = newy
        
    def getPoint(self):
        return self.x, self.y
    
    def distanceFromOrigin(self):
        return sqrt(pow(self.x, 2) + pow(self.y, 2))
    
    def distanceToOther(self, other):
        dx = abs(self.x - other.x)
        dy = abs(self.y - other.y)
        return round(sqrt(pow(dx, 2) + pow(dy, 2)), 2)
    
    def translate(self, dx, dy):
        self.x += dx
        self.y += dy
        

In [2]:
# define the point sets function
# create multiple points in random
# the point structure follows the Point ADT class
import random
def pointSet(n):   # make n points into the pointset
    ps = list()
    for i in range(n):   # create n points
        x = random.randint(0, 100)
        y = random.randint(0, 100)
        
        notSame = True 
        for j in ps:     # check if the created point is already exist.
                         # if yes, re-create a new point.
            if x == j.x and y == j.y:
                i -= 1
                notSame = False
                break
        if notSame == True:
            ps.append(Point(x, y))
    return ps
                                 

## Question 2:
Implement an ADT class for Partition (union-find). It must support the operations for
generating a new partition (i.e. a set of sets), for generating a new set within the partition,
for merging two sets (union) and for finding a set to which an element belongs (find). You
should use the tree-based implementation. If you need a refresher regarding the Partition
ADT, consult the lecture notes for spanning trees. Wikipedia also has a good and compact
explanation here. (*Note: In the pracs, we implemented the union-find procedure. You now
have to implement it as an ADT class)

In [3]:
# Create a non-direction graph structure
class Graph:
    def __init__(self, ps):
        self.vertix = ps
        self.edge = list()
        
        # generate edges between every two points
        for i in range(len(self.vertix)-1):
            for j in range(i+1, len(self.vertix)):
                self.edge.append((ps[i].distanceToOther(ps[j]), ps[i], ps[j]))
    
    def getVertix(self):
        return self.vertix
    
    def getEdge(self):
        return self.edge
    
    def getVertixNum(self):
        return len(self.vertix)
    
    def getEdgeNum(self):
        return len(self.edge)
    
    # Manually defined edges sorting to solve the same distance sorting problem         
    def sortEdge(self):
        if len(self.edge) > 1:
            for i in range(len(self.edge)-1):
                for j in range(i+1, len(self.edge)):
                    if self.edge[i][0] > self.edge[j][0]:
                        self.edge[i], self.edge[j] = self.edge[j], self.edge[i]
                
                

In [4]:
# Data structure Partition
# in the tree-based implementation, each point is regarded as a tree
# with their parent linked.
class Partition:
    def __init__(self, ps):
        self.parent = dict()
        self.rank = dict()
        self.graph = Graph(ps)   # store point sets into graph structure
        self.spanEdge = list()   # store the spanning edges
        self.pointSets = list()  # make a set of sets
        self.ps = ps
        
        for p in self.ps:   # initialize the set of sets
            s = set()
            s.add(p)
            self.pointSets.append(s)
       
    
    def getGraph(self):
        return self.graph
    
    def getParent(self):
        return self.parent
    
    def getRank(self):
        return self.rank
    
    def getSpanEdge(self):
        return self.spanEdge
        
    def getPointSets(self):
        return self.pointSets
    
    def makeSet(self, x):
        if x in self.graph.getVertix():
            self.parent[x] = x
            self.rank[x] = 0
            return True
        else:
            return False
        
    ####### Question 2, Non-path compression find method ##########
    def find(self, x):    # find root of x
        if x in self.graph.getVertix():
            while x != self.parent[x]:
                x = self.parent[x]
            return x
        else:
            return None

    ############ Answer of Question 4 ############
    # path compression in find method: make the parent of x as its root.
    def findPC(self, x): 
        if x in self.graph.getVertix():
            if x != self.parent[x]:
                self.parent[x] = self.findPC(self.parent[x])
            return x
        else:
            return None
    
    def union(self, x, y):
        A = self.find(x)
        B = self.find(y)
        if self.rank[A] > self.rank[B]:
            self.parent[B] = A
        else:
            self.parent[A] = B
            if self.rank[A] == self.rank[B]:
                self.rank[B] += 1
        # combine the sets, to make sure all the node in same tree is also in same set
        indx = -1
        indy = -1
        for i in range(len(self.pointSets)):
            if x in self.pointSets[i]:
                indx = i    # store the set list index of x
            if y in self.pointSets[i]:
                indy = i    # store the set list index of y
        pSetX = self.pointSets[indx]
        pSetY = self.pointSets.pop(indy)
        self.pointSets[self.pointSets.index(pSetX)] = self.pointSets[self.pointSets.index(pSetX)] | pSetY
                        
    def kruskal(self):
        self.graph.sortEdge()
        edges = self.graph.getEdge()
        for v in self.graph.getVertix():
            self.makeSet(v)
        for i in edges:
            weight, v1, v2 = i
            if self.find(v1) != self.find(v2):
                self.union(v1, v2)
                self.spanEdge.append(i)
        return self.spanEdge
    
    ############ Question 5, k-cluster kruskal #############
    def kClusterKruskal(self, k):  # define the k cluster kruskal
        self.graph.sortEdge()
        edges = self.graph.getEdge()
        for v in self.graph.getVertix():
            self.makeSet(v)
        for i in edges:
            if len(self.pointSets) == k:   # use the number of sets to determine k-cluster
                break
            else:
                weight, v1, v2 = i
                if self.find(v1) != self.find(v2):
                    self.union(v1, v2)
                    self.spanEdge.append(i)
        return self.spanEdge
    
    ############# Question 6 ############
    def inSameCluster(self, x, y):
        if x in self.graph.getVertix() and y in self.graph.getVertix():
            if self.findPC(x) == self.findPC(y):
                return True
            else:
                return False
        else:
            return False
    
    ############ Question 7, Denn Index ############# 
    # DennIndex = min distance between village / max distance in one village
    def dennIndex(self):
        if len(self.pointSets) == 1:
            return 0
        else:
            # minimum distance between villages
            self.graph.sortEdge()
            graphEdges = self.graph.getEdge()
            maxSpanEdges = self.spanEdge[-1]
            ind = graphEdges.index(maxSpanEdges)
            minBetVil = 0
            while True:
                ind += 1
                weight, v1, v2 = graphEdges[ind]
                if self.findPC(v1) != self.findPC(v2):
                    minBetVil = weight
                    break
            # maximum distance in villages
            maxInVil = 0
            for i in self.pointSets:
                if len(i) > 1:
                    village = Graph(list(i))
                    village.sortEdge()
                    weight, v1, v2 = village.getEdge()[-1]
                    if weight > maxInVil:
                        maxInVil = weight
                
            dunn = minBetVil/maxInVil
            return dunn
    

## Question 3:
Using the Partition ADT and test data function that you wrote for the first two tasks,
implement the clustering procedure described above. If you did not manage to implement
the tree structure or the forest structure in the earlier tasks, you may do so using the simple
list-based union-find implementation, but this will only attract partial marks.

In [5]:
# Create a point set with 20 points in there
ps = pointSet(20)
for i in ps:
    print("point:", i)
print("\nGenerate", len(ps), "points in total.")

point: (4, 43)
point: (27, 37)
point: (3, 88)
point: (4, 26)
point: (2, 0)
point: (37, 50)
point: (0, 31)
point: (45, 26)
point: (48, 89)
point: (76, 39)
point: (41, 82)
point: (51, 17)
point: (24, 76)
point: (37, 70)
point: (91, 27)
point: (51, 87)
point: (7, 14)
point: (4, 38)
point: (86, 92)
point: (14, 51)

Generate 20 points in total.


In [6]:
sets = Partition(ps)
totalWeight = 0
edges = sets.kruskal()  # cluster into one spanning tree
for i in edges:  
    weight, v1, v2 = i
    print("The two Vertices:", v1, "and", v2, "   The Weight is:", weight)
    totalWeight += weight
print("\nTotal Weight is:", round(totalWeight, 2), "\nNumber of edges:", len(edges))
print("Number of spanning tree:", len(sets.getPointSets()))

The two Vertices: (48, 89) and (51, 87)    The Weight is: 3.61
The two Vertices: (4, 43) and (4, 38)    The Weight is: 5.0
The two Vertices: (4, 26) and (0, 31)    The Weight is: 6.4
The two Vertices: (0, 31) and (4, 38)    The Weight is: 8.06
The two Vertices: (48, 89) and (41, 82)    The Weight is: 9.9
The two Vertices: (45, 26) and (51, 17)    The Weight is: 10.82
The two Vertices: (4, 26) and (7, 14)    The Weight is: 12.37
The two Vertices: (41, 82) and (37, 70)    The Weight is: 12.65
The two Vertices: (4, 43) and (14, 51)    The Weight is: 12.81
The two Vertices: (24, 76) and (37, 70)    The Weight is: 14.32
The two Vertices: (2, 0) and (7, 14)    The Weight is: 14.87
The two Vertices: (27, 37) and (37, 50)    The Weight is: 16.4
The two Vertices: (27, 37) and (14, 51)    The Weight is: 19.1
The two Vertices: (76, 39) and (91, 27)    The Weight is: 19.21
The two Vertices: (37, 50) and (37, 70)    The Weight is: 20.0
The two Vertices: (27, 37) and (45, 26)    The Weight is: 21.1


## Question 4:
Extend your forest-based Partition ADT class from above with path compression.

###### - Answer:
* Already wrote in Question 2: Partition ADT
* Method findPC(self, x)

## Question 5:
Ultimately, we are aiming to find k-clusters i.e. we want to the electricity providers to see
the structure of the micro-grid if they decide to commission k micro-grids such that all the
nodes are covered in the vicinity of these sub-stations. Extend your implementation such
that it stops when k clusters have been achieved and return those clusters, where k is a
user-definable parameter.

In [7]:
kClusterSets = Partition(ps)
spanEdges = kClusterSets.kClusterKruskal(6)
print("Total point sets:", [p.getPoint() for p in ps])
for i in spanEdges:
    print("Weight:", i[0], " Between:", i[1].__str__(), "and", i[2].__str__())
print("\nEdges number:", len(spanEdges), "\nNumber of villages:", len(kClusterSets.getPointSets()))

Total point sets: [(4, 43), (27, 37), (3, 88), (4, 26), (2, 0), (37, 50), (0, 31), (45, 26), (48, 89), (76, 39), (41, 82), (51, 17), (24, 76), (37, 70), (91, 27), (51, 87), (7, 14), (4, 38), (86, 92), (14, 51)]
Weight: 3.61  Between: (48, 89) and (51, 87)
Weight: 5.0  Between: (4, 43) and (4, 38)
Weight: 6.4  Between: (4, 26) and (0, 31)
Weight: 8.06  Between: (0, 31) and (4, 38)
Weight: 9.9  Between: (48, 89) and (41, 82)
Weight: 10.82  Between: (45, 26) and (51, 17)
Weight: 12.37  Between: (4, 26) and (7, 14)
Weight: 12.65  Between: (41, 82) and (37, 70)
Weight: 12.81  Between: (4, 43) and (14, 51)
Weight: 14.32  Between: (24, 76) and (37, 70)
Weight: 14.87  Between: (2, 0) and (7, 14)
Weight: 16.4  Between: (27, 37) and (37, 50)
Weight: 19.1  Between: (27, 37) and (14, 51)
Weight: 19.21  Between: (76, 39) and (91, 27)

Edges number: 14 
Number of villages: 6


In [8]:
for i in ps:
    print(i, "has root:", kClusterSets.getParent()[i])

(4, 43) has root: (4, 38)
(27, 37) has root: (37, 50)
(3, 88) has root: (3, 88)
(4, 26) has root: (0, 31)
(2, 0) has root: (4, 38)
(37, 50) has root: (4, 38)
(0, 31) has root: (4, 38)
(45, 26) has root: (51, 17)
(48, 89) has root: (51, 87)
(76, 39) has root: (91, 27)
(41, 82) has root: (51, 87)
(51, 17) has root: (51, 17)
(24, 76) has root: (51, 87)
(37, 70) has root: (51, 87)
(91, 27) has root: (91, 27)
(51, 87) has root: (51, 87)
(7, 14) has root: (4, 38)
(4, 38) has root: (4, 38)
(86, 92) has root: (86, 92)
(14, 51) has root: (4, 38)


In [9]:
for i in kClusterSets.getPointSets():
    print("A village:", [j.__str__() for j in i])

A village: ['(14, 51)', '(7, 14)', '(2, 0)', '(4, 26)', '(0, 31)', '(27, 37)', '(37, 50)', '(4, 38)', '(4, 43)']
A village: ['(3, 88)']
A village: ['(45, 26)', '(51, 17)']
A village: ['(76, 39)', '(91, 27)']
A village: ['(24, 76)', '(37, 70)', '(51, 87)', '(41, 82)', '(48, 89)']
A village: ['(86, 92)']


## Question 6:
Implement functions/methods to let the user query whether two given points (nodes)
belong to the same cluster (micro-grid).

In [10]:
p1 = ps[3]
p2 = ps[5]
print("The points,", p1, "and", p2, "have same root:")
kClusterSets.inSameCluster(p1, p2)

The points, (4, 26) and (37, 50) have same root:


False

## Question 7:
Define a function to compute the Dunn index, a measure for the quality of the clustering.

In [11]:
print("Denn Index is:", kClusterSets.dennIndex())

Denn Index is: 0.3277076847452073


## Question 8:
Compare the forest that you have generated for the k-clustering to the full minimum cost
spanning tree. Give a brief, but precise characterization of the sets of edges that are in the
MCST but not in the forest of the k-clustering.

###### - Answer:
The aim of k-clustering is to decide k number of villages, and each village has a micro-grad. The sets of edges which are in MCST but not in k-clustering are the lines connected between different micro-grads in villages. Using MCST is to make the villages connecting lines have lowest cost between every two villages.

## Bonus:

In [12]:
# for This bonus, I use Bokeh in Anaconda Jupyter Notebook
# install Bokeh in Anaconda prompt: conda install bokeh
# install Bokehlab which is used in Jupyter: conda install -c conda-forge jupyterlab
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, BoxSelectTool #For enabling tools
from ipywidgets import interact
import numpy as np
from bokeh.plotting import figure


In [13]:
x = list()
y = list()
for i in ps:
    x.append(i.getPoint()[0])
    y.append(i.getPoint()[1])

In [14]:
# MCST graph
output_notebook()
TOOLS = [BoxSelectTool(), HoverTool()]
p = figure(title="Spanning Tree", plot_width=800, plot_height=800, tools=TOOLS)
p.circle(x, y, size=10, color="black")
t = show(p, notebook_handle=True)
for i in sets.getSpanEdge():
    v1, v2 = i[1], i[2]
    x1, y1 = v1.getPoint()
    x2, y2 = v2.getPoint()
    xx = [x1, x2]
    yy = [y1, y2]
    p.line(xx, yy, line_width=2, color="orange")
    push_notebook(handle=t)


Loading BokehJS ...

In [15]:
# k-cluster graph
output_notebook()
TOOLS = [BoxSelectTool(), HoverTool()]
p1 = figure(title="k-cluster", plot_width=800, plot_height=800, tools=TOOLS)
p1.circle(x, y, size=10, color="black")
t1 = show(p1, notebook_handle=True)
for ii in kClusterSets.getSpanEdge():
    v1, v2 = ii[1], ii[2]
    x1, y1 = v1.getPoint()
    x2, y2 = v2.getPoint()
    xx = [x1, x2]
    yy = [y1, y2]
    p1.line(xx, yy, line_width=2, color="red")
    push_notebook(handle=t1)

Loading BokehJS ...